In [1]:
import pandas as pd
import os
import gc
import sqlite3

# Universe Definition

1. I downloaded PIT Russell 3000 consituents from the following link.
https://www.ishares.com/us/products/239714/ishares-russell-3000-etf
2. Once I downloaded a seperate file for each day available, I found the first date that an ISIN was in the russell 3000.
3. A stock that enteres the russell 3000 is in the universe for all time, even if it later falls out of the Russell 3000

In [2]:
# Get list of all files in files = list(os.listdir())
Universe = pd.DataFrame()
files = list(os.listdir('./Russell3000'))

for file in files:
    if file == '.DS_Store':
        continue
    df = pd.read_csv('./Russell3000/' + file,skiprows = 10)
    df1 = pd.read_csv('./Russell3000/' + file,skiprows = 2,nrows = 1,names = ["name","date"],parse_dates = ['date'])
    df['date'] = df1['date'][0]
    Universe = pd.concat([Universe,df[['ISIN','date']]])


Universe = Universe.groupby(by = 'ISIN').min().reset_index()
Universe.head()

,ISIN,date
0,-,2006-09-29
1,1188001,2006-09-29
2,2576101,2006-09-29
3,3381001,2006-09-29
4,3599502,2014-03-31


# Getting Mapper File from CID - ISIN for our Universe

In [53]:
fs_ref = pd.read_csv('fs_company_reference_daily_2019-10-09.csv.gz', parse_dates = ['start_date','end_date'])
fs_ref = fs_ref[fs_ref.primary_flag == True]
fs_ref = fs_ref[fs_ref['isin'].isin(Universe.ISIN)]
print(len(fs_ref))

Universe_Mapper = fs_ref[['company_id','isin','start_date','end_date']]

Universe_Mapper['start_date'].fillna('1900-01-01', inplace = True)
Universe_Mapper['end_date'].fillna('2100-01-01', inplace = True)
Universe_Mapper.drop_duplicates(inplace = True)

Universe_Mapper = pd.merge(left = Universe_Mapper, right = Universe, how = 'left',left_on = 'isin',right_on = 'ISIN')
x=0
for index, row in Universe_Mapper.iterrows():
    if row.start_date > row.date:
        continue
    else:
        row.start_date = row.date

    
print(len(Universe_Mapper))
Universe_Mapper.head(100)

15838


/Users/iflath/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


TypeError: Cannot compare type 'Timestamp' with type 'str'

In [9]:
conn = sqlite3.connect(':memory:')
pd.read_csv('Raw_Quant_Company_Daily_Unique_Active_PP_2019-10-17.zip').to_sql('UA','')
UA = pd.merge(UA,, how = 'left', left_on = 

SyntaxError: invalid syntax (<ipython-input-9-afb9c50f78dc>, line 3)